# Chapter 14: Joining and Concatenating

In [1]:
import polars as pl
pl.show_versions()

--------Version info---------
Polars:               0.20.31
Index type:           UInt32
Platform:             macOS-12.5-arm64-arm-64bit
Python:               3.11.9 (main, Apr  2 2024, 16:11:47) [Clang 14.0.0 (clang-1400.0.29.202)]

----Optional dependencies----
adbc_driver_manager:  0.8.0
cloudpickle:          3.0.0
connectorx:           0.3.2
deltalake:            0.15.0
fastexcel:            0.9.1
fsspec:               2023.12.2
gevent:               23.9.1
hvplot:               0.9.2
matplotlib:           3.8.4
nest_asyncio:         1.6.0
numpy:                1.26.4
openpyxl:             3.1.2
pandas:               2.2.2
pyarrow:              14.0.2
pydantic:             2.5.3
pyiceberg:            0.5.1
pyxlsb:               <not installed>
sqlalchemy:           2.0.25
torch:                <not installed>
xlsx2csv:             0.8.2
xlsxwriter:           3.2.0


## Joining

### Join Strategies

In [4]:
import polars as pl

df_left = pl.DataFrame({
    "key": ["A", "B", "C", "D"],
    "value": [1, 2, 3, 4]
})

df_right = pl.DataFrame({
    "key": ["B", "C", "D", "E"],
    "value": [5, 6, 7, 8]
})


In [5]:
df_left.join(df_right, on="key", how="inner")

key,value,value_right
str,i64,i64
"""B""",2,5
"""C""",3,6
"""D""",4,7


In [6]:
df_left.join(df_right, on="key", how="outer", suffix="_other")

/var/folders/3d/ldmd42_x7zx8d8xqy0fkhmgc0000gn/T/ipykernel_80304/2826378691.py:1: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  df_left.join(df_right, on="key", how="outer", suffix="_other")


key,value,key_other,value_other
str,i64,str,i64
"""B""",2,"""B""",5
"""C""",3,"""C""",6
"""D""",4,"""D""",7
null,null,"""E""",8
"""A""",1,null,null


In [7]:
df_left.join(df_right, on="key", how="left")

/var/folders/3d/ldmd42_x7zx8d8xqy0fkhmgc0000gn/T/ipykernel_80304/1462491990.py:1: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_left.join(df_right, on="key", how="left")


key,value,value_right
str,i64,i64
"""A""",1,null
"""B""",2,5
"""C""",3,6
"""D""",4,7


In [8]:
df_left.join(df_right, how="cross")

key,value,key_right,value_right
str,i64,str,i64
"""A""",1,"""B""",5
"""A""",1,"""C""",6
"""A""",1,"""D""",7
"""A""",1,"""E""",8
"""B""",2,"""B""",5
…,…,…,…
"""C""",3,"""E""",8
"""D""",4,"""B""",5
"""D""",4,"""C""",6


In [9]:
df_left.join(df_right, on="key", how="semi")

key,value
str,i64
"""B""",2
"""C""",3
"""D""",4


In [10]:
df_left.join(df_right, on="key", how="anti")

key,value
str,i64
"""A""",1


### Joining on Multiple Columns

In [12]:
df_left = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "Dave"],
    "city": ["NY", "LA", "NY", "SF"],
    "age": [25, 30, 35, 40]
})

df_right = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "Dave"],
    "city": ["NY", "LA", "NY", "Chicago"],
    "department": ["Finance", "Marketing", "Engineering", "Operations"]
})

df_left.join(df_right, on=["name", "city"], how="inner")

name,city,age,department
str,str,i64,str
"""Alice""","""NY""",25,"""Finance"""
"""Bob""","""LA""",30,"""Marketing"""
"""Charlie""","""NY""",35,"""Engineering"""


### Validation

In [14]:
df_employees = pl.DataFrame({
    "employee_id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie", "Dave"],
    "department_id": [10, 10, 30, 10],
})

df_departments = pl.DataFrame({
    "department_id": [10, 20, 30],
    "department_name": ["Information Technology", "Finance", "Human Resources"],
})

df_employees.join(
    df_departments,
    on="department_id",
    how="left",
    validate="m:1"
)

/var/folders/3d/ldmd42_x7zx8d8xqy0fkhmgc0000gn/T/ipykernel_80304/2858785018.py:12: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_employees.join(


employee_id,name,department_id,department_name
i64,str,i64,str
1,"""Alice""",10,"""Information Technology"""
2,"""Bob""",10,"""Information Technology"""
3,"""Charlie""",30,"""Human Resources"""
4,"""Dave""",10,"""Information Technology"""


In [15]:
df_departments = pl.DataFrame({
    "department_id": [10, 20, 10],
    "department_name": ["Information Technology", "Finance", "Human Resources"],
})

df_employees.join(
    df_departments,
    on="department_id",
    how="left",
    validate="m:1"
)

/var/folders/3d/ldmd42_x7zx8d8xqy0fkhmgc0000gn/T/ipykernel_80304/87466152.py:6: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_employees.join(


ComputeError: the join keys did not fulfil m:1 validation

## Inexact Joining

In [17]:
df_left = pl.DataFrame({
    "int_id": [5, 10],
    "value": ["1", "2"]
})

df_right = pl.DataFrame({
    "int_id": [4, 7, 12],
    "value": [1, 2, 3]
})

df_left.join_asof(df_right, on="int_id", tolerance=3)

int_id,value,value_right
i64,str,i64
5,"""1""",1
10,"""2""",2


In [18]:
df_left = df_left.set_sorted("int_id")
df_right = df_right.set_sorted("int_id")

df_left.join_asof(df_right, on="int_id")

int_id,value,value_right
i64,str,i64
5,"""1""",1
10,"""2""",2


In [19]:
df_right = df_right.rename({"int_id": "int_id_right"})

df_left.join_asof(
    df_right,
    left_on="int_id",
    right_on="int_id_right",
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""1""",4,1
10,"""2""",7,2


In [20]:
df_left.join_asof(
    df_right,
    left_on="int_id",
    right_on="int_id_right",
    tolerance=3,
    strategy="backward"
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""1""",4,1
10,"""2""",7,2


In [21]:
df_left.join_asof(
    df_right,
    left_on="int_id",
    right_on="int_id_right",
    tolerance=3,
    strategy="forward"
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""1""",7,2
10,"""2""",12,3


In [22]:
df_left.join_asof(
    df_right,
    left_on="int_id",
    right_on="int_id_right",
    tolerance=3,
    strategy="nearest"
)

int_id,value,int_id_right,value_right
i64,str,i64,i64
5,"""1""",4,1
10,"""2""",12,3


### Additional Finetuning with `tolerance` and `by`

### Use Case: Marketing Campaign Attribution

In [25]:
marketing_lf = pl.scan_csv("data/marketing use case/marketing_campaigns.csv")
marketing_lf.fetch(1)

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/marketing_campaigns.csv

In [26]:
marketing_lf.select(pl.col("Product Type").unique()).collect()

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/marketing_campaigns.csv

This error occurred with the following context stack:
	[1] 'csv scan' failed
	[2] 'select' input failed to resolve


In [27]:
sales_lf = pl.scan_csv("data/marketing use case/sales_data.csv")
sales_lf.fetch(1)

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/sales_data.csv

In [28]:
sales_lf = sales_lf.with_columns(
    pl.col("Sale Date")
    .str.to_datetime("%Y-%m-%d %H:%M:%S%.f")
    .cast(pl.Datetime("us")),
)
marketing_lf = marketing_lf.with_columns(
    pl.col("Campaign Date").str.to_datetime("%Y-%m-%d %H:%M:%S"),
)

sales_with_campaign_df = (
    sales_lf
    .sort("Sale Date")
    .join_asof(
        marketing_lf
        .sort("Campaign Date"),
        left_on="Sale Date",
        right_on="Campaign Date",
        by="Product Type",
        strategy="backward",
        tolerance="60d"
    )
    .collect()
)
sales_with_campaign_df

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/sales_data.csv

This error occurred with the following context stack:
	[1] 'csv scan' failed
	[2] 'with_columns' input failed to resolve
	[3] 'sort' input failed to resolve
	[4] 'join left' input failed to resolve


In [29]:
(
    sales_with_campaign_df
    .group_by("Product Type", "Campaign Name")
    .agg(pl.col("Quantity").mean())
    .sort("Product Type", "Campaign Name")
)

NameError: name 'sales_with_campaign_df' is not defined

In [30]:
marketing_lf.filter(pl.col("Product Type") == "Books").collect()

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/marketing_campaigns.csv

This error occurred with the following context stack:
	[1] 'csv scan' failed
	[2] 'with_columns' input failed to resolve
	[3] 'filter' input failed to resolve


In [31]:
(
    sales_lf
    .filter(
        (pl.col("Product Type") == "Books") &
        (
            pl.col("Sale Date") >
            pl.lit("2023-12-31 21:00:00").str.to_datetime()
        )
    )
    .collect()
)

FileNotFoundError: No such file or directory (os error 2): data/marketing use case/sales_data.csv

This error occurred with the following context stack:
	[1] 'csv scan' failed
	[2] 'with_columns' input failed to resolve
	[3] 'filter' input failed to resolve


## Vertical and Horizontal Concatenation

In [33]:
df1 = pl.DataFrame({
    "id": [1, 2, 3],
    "value": ["a", "b", "c"],
})
df2 = pl.DataFrame({
    "id": [4, 5],
    "value": ["d", "e"],
})
pl.concat([df1,df2], how="vertical")

id,value
i64,str
1,"""a"""
2,"""b"""
3,"""c"""
4,"""d"""
5,"""e"""


In [34]:
df1 = pl.DataFrame({
    "id": [1, 2, 3],
    "value": ["a", "b", "c"],
})
df2 = pl.DataFrame({
    "value2": ["x", "y"],
})
pl.concat([df1,df2], how="horizontal")

id,value,value2
i64,str,str
1,"""a""","""x"""
2,"""b""","""y"""
3,"""c""",null


In [35]:
df1 = pl.DataFrame({
    "id": [1, 2, 3],
    "value": ["a", "b", "c"],
})
df2 = pl.DataFrame({
    "value": ["d", "e"],
    "value2": ["x", "y"],
})
pl.concat([df1,df2], how="diagonal")

id,value,value2
i64,str,str
1,"""a""",null
2,"""b""",null
3,"""c""",null
null,"""d""","""x"""
null,"""e""","""y"""


In [36]:
df1 = pl.DataFrame({
    "id": [1, 2, 3],
    "value": ["a", "b", "c"],
})
df2 = pl.DataFrame({
    "value": ["a", "c", "d"],
    "value2": ["x", "y", "z"],
})
pl.concat([df1,df2], how="align")

id,value,value2
i64,str,str
1,"""a""","""x"""
2,"""b""",null
3,"""c""","""y"""
null,"""d""","""z"""


In [37]:
df1 = pl.DataFrame({
    "id": [1, 2, 3],
    "value": ["a", "b", "c"],
})
df2 = pl.DataFrame({
    "id": [4.0, 5.0],
    "value": [1, 2],
})
pl.concat([df1,df2], how="vertical")

SchemaError: type Float64 is incompatible with expected type Int64

In [38]:
pl.concat([df1,df2], how="vertical_relaxed")

id,value
f64,str
1.0,"""a"""
2.0,"""b"""
3.0,"""c"""
4.0,"""1"""
5.0,"""2"""


In [39]:
df1 = pl.DataFrame({
    "id": [1, 2, 2],
    "value": ["a", "c", "b"],
})
df2 = pl.DataFrame({
    "id": [2, 2],
    "value": ["x", "y"],
})
pl.align_frames(df1,df2, on="id")

[shape: (5, 2)
 ┌─────┬───────┐
 │ id  ┆ value │
 │ --- ┆ ---   │
 │ i64 ┆ str   │
 ╞═════╪═══════╡
 │ 1   ┆ a     │
 │ 2   ┆ c     │
 │ 2   ┆ c     │
 │ 2   ┆ b     │
 │ 2   ┆ b     │
 └─────┴───────┘,
 shape: (5, 2)
 ┌─────┬───────┐
 │ id  ┆ value │
 │ --- ┆ ---   │
 │ i64 ┆ str   │
 ╞═════╪═══════╡
 │ 1   ┆ null  │
 │ 2   ┆ x     │
 │ 2   ┆ y     │
 │ 2   ┆ x     │
 │ 2   ┆ y     │
 └─────┴───────┘]

In [40]:
df1 = pl.DataFrame({
    "id": [1, 2],
    "value": ["a", "b"],
})
df2 = pl.DataFrame({
    "id": [3, 4],
    "value": ["c", "d"],
})
df1.vstack(df2)

id,value
i64,str
1,"""a"""
2,"""b"""
3,"""c"""
4,"""d"""


In [41]:
df1 = pl.DataFrame({
    "id": [1, 2],
    "value": ["a", "b"],
})
df2 = pl.DataFrame({
    "value2": ["x", "y"],
})
df1.hstack(df2)

id,value,value2
i64,str,str
1,"""a""","""x"""
2,"""b""","""y"""


In [42]:
s1 = pl.Series("a", [1, 2])
s2 = pl.Series("b", [3, 4])
s1.append(s2)

a
i64
1
2
3
4


In [43]:
df1 = pl.DataFrame({
    "id": [1, 2],
    "value": ["a", "b"],
})
df2 = pl.DataFrame({
    "id": [3, 4],
    "value": ["c", "d"],
})
df1.extend(df2)

id,value
i64,str
1,"""a"""
2,"""b"""
3,"""c"""
4,"""d"""


## Conclusion